<a href="https://colab.research.google.com/github/Hanamoongit/Hanamoongit/blob/main/google_colab_notebooks/Spark_Logs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code-Guide: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html

Spark Vorteile
* Gegenüber Pandas: Spark hält Daten nach Möglichkeit im Arbeitsspeicher, reduziert Festplatten-I/O und beschleunigt wiederholte Abfragen. Pandas liest und schreibt standardmässig auf die Festplatte.
* Spark erstellt einen optimierten Ausführungsplan erst beim Action-Aufruf, vermeidet unnötige Zwischenschritte und verbessert somit die Gesamtperformance.
* Spark repliziert Partitionen und kann bei Node-Ausfällen fehlende Tasks automatisch neu starten.
* Spark verteilt Workloads automatisch auf alle Workernodes im Cluster.

# Analyse der Top 10 Hostnames nach Anzahl der Zugriffe
### Google Colab Example

# Big Data - Map Reduce mit Pyton - Textmining
### Google Colab Example

In [ ]:
# --- 0. Drive mounten, damit Colab auf deine Drive-Dateien zugreifen kann ---
from google.colab import drive
drive.mount('/content/drive')
# Ich werde dazu aufgefordert, Colab den Zugriff auf mein Google-Konto zu erlauben.

# Pfad zu meiner Datei in Google Drive:
# „Meine Ablage/BDS Bibliotheken/logs.csv“ entspricht im Colab-Mount:
drive_path = "/content/drive/My Drive/BDS Bibliotheken/logs.csv"

# --- 1. Spark-Session initialisieren (hier lokal) ---
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = (
    SparkSession.builder
        .appName("LogAnalysis")
        .master("local[*]")                       # lokale Ausführung
        .config("spark.ui.showConsoleProgress", "true")
        .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")            # weniger Lärm im Log

# --- 2. Log-Datei aus Google Drive einlesen ---
df = (
    spark.read
         .option("header", "true")                # erste Zeile = Spaltennamen
         .option("inferSchema", "true")           # Typen automatisch ermitteln
         .csv(drive_path)                         # hier dein Drive-Pfad
)

# --- 3. Zugriffe pro Hostname zählen ---
hostname_counts = df.groupBy("hostname").count()

# --- 4. Sortierung nach Anfragezahl (absteigend) ---
sorted_counts = hostname_counts.orderBy(col("count").desc())

# --- 5. Top-10 Hostnames auswählen ---
top10_hosts = sorted_counts.limit(10)

# --- 6. Ergebnisse anzeigen ---
top10_hosts.show(truncate=False)

# --- 7. Spark-Session beenden ---
spark.stop()

Mounted at /content/drive
+-----------------------------------+-----+
|hostname                           |count|
+-----------------------------------+-----+
|int0.client.access.fanaptelecom.net|101  |
|unknown.puregig.net                |31   |
|server2us.getmyip.co               |5    |
|swe-net-ip.as51430.net             |5    |
|NULL                               |4    |
|zmap.sorengard.com                 |4    |
|host.coloup.com                    |4    |
|client.pool.myshanet.net           |3    |
|pcomtelecom.net                    |3    |
|no-mans-land.m247.com              |3    |
+-----------------------------------+-----+



Weitere Schritte, die möglich wären (für eine End-to-End-Pipeline):
* Bereinigung der IP-Adressen, Entfernen ungültiger oder leerer Einträge
* Erkennung von Anomalien oder potenziell bösartigen Traffic
* Speicherung der Resultate im Data Warehouse
* Automatisierung des Workflows mit Airflow oder Oozie
* ...